## Cargamos el primer dataset de google con la metadata de los restaurantes

In [1]:
from google.colab import auth
auth.authenticate_user()

# bibliotecas
from google.cloud import storage
import pandas as pd
import io

# cliente de almacenamiento
client = storage.Client()

# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'MetaDataGoogleRestaurants.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_md = pd.read_csv(io.BytesIO(file_content))

df_md['platform'] = 'Google'
df_md.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,platform
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,'Korean restaurant',4.4,18,Google
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,'Restaurant',4.5,18,Google
2,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,41.616079,-93.865487,"'Bakery', 'Health food restaurant'",4.7,21,Google
3,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,'Restaurant',4.0,8,Google
4,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,21.637796,-157.920714,'Restaurant',4.4,18,Google


## Cambiamos las columnas que no nos sirven y generamos el business_id desde latitud y longitud

In [10]:
df_bgu = df_md.drop(columns = ['gmap_id'])
df_bgu['latitude'] = df_bgu['latitude'].astype(str)
df_bgu['longitude'] = df_bgu['longitude'].astype(str)
df_bgu['business_id'] = df_bgu['latitude'] + df_bgu['longitude']
df_bgu.rename(columns={'avg_rating':'stars'}, inplace = True)
df_bgu.rename(columns={'num_of_reviews':'review_count'}, inplace = True)
df_bgu.rename(columns={'category':'categories'}, inplace = True)
df_bgu.drop_duplicates(inplace=True)
df_bgu.head()

,name,address,latitude,longitude,categories,stars,review_count,platform,business_id
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",34.0580917,-118.2921295,'Korean restaurant',4.4,18,Google,34.0580917-118.2921295
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",33.9164023,-118.010855,'Restaurant',4.5,18,Google,33.9164023-118.010855
2,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",41.616079,-93.865487,"'Bakery', 'Health food restaurant'",4.7,21,Google,41.616079-93.865487
3,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",39.9402929,-75.150923,'Restaurant',4.0,8,Google,39.9402929-75.150923
4,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",21.6377957,-157.9207142,'Restaurant',4.4,18,Google,21.6377957-157.9207142


In [8]:
df_bgu.shape

(210953, 9)

## Cargamos el archivo business de YELP

In [3]:
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'business.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_by = pd.read_csv(io.BytesIO(file_content))

df_by['platform'] = 'Yelp'
df_by.head()


,business_id,name,address,city,state,latitude,longitude,stars,review_count,categories,platform
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Yelp
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Yelp
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,38.565165,-90.321087,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",Yelp
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,36.208102,-86.768170,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Yelp
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,MO,27.955269,-82.456320,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",Yelp


## Modificamos columnas y creamos business_id a partir de latitud y longitud

In [4]:
df_byu = df_by.drop(columns = ['business_id','city','state'])
df_byu['latitude'] = df_byu['latitude'].astype(str)
df_byu['longitude'] = df_byu['longitude'].astype(str)
df_byu['business_id'] = df_byu['latitude'] + df_byu['longitude']
df_byu.drop_duplicates(inplace=True)
df_byu.head()

,name,address,latitude,longitude,stars,review_count,categories,platform,business_id
0,St Honore Pastries,935 Race St,39.9555052,-75.1555641,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Yelp,39.9555052-75.1555641
1,Sonic Drive-In,615 S Main St,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Yelp,36.269593-87.058943
2,Tsevi's Pub And Grill,8025 Mackenzie Rd,38.5651648,-90.3210868,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",Yelp,38.5651648-90.3210868
3,Sonic Drive-In,2312 Dickerson Pike,36.2081024,-86.7681696,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Yelp,36.2081024-86.7681696
4,Vietnamese Food Truck,NaN,27.9552692,-82.4563199,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",Yelp,27.9552692-82.4563199


## Generamos el dataset unificado

In [19]:
# Unificar los dataframes uno debajo del otro
df_business_unificado = pd.concat([df_byu, df_bgu], ignore_index=True)

df_business_unificado.drop_duplicates(subset='business_id', keep='first', inplace= True)
df_business_unificado.reset_index(inplace=True, drop=True)
df_business_unificado

,name,address,latitude,longitude,stars,review_count,categories,platform,business_id
0,St Honore Pastries,935 Race St,39.9555052,-75.1555641,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Yelp,39.9555052-75.1555641
1,Sonic Drive-In,615 S Main St,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Yelp,36.269593-87.058943
2,Tsevi's Pub And Grill,8025 Mackenzie Rd,38.5651648,-90.3210868,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",Yelp,38.5651648-90.3210868
3,Sonic Drive-In,2312 Dickerson Pike,36.2081024,-86.7681696,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Yelp,36.2081024-86.7681696
4,Vietnamese Food Truck,NaN,27.9552692,-82.4563199,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",Yelp,27.9552692-82.4563199
...,...,...,...,...,...,...,...,...,...
254573,Millsite Inn,"Millsite Inn, 44365 CO-72, Ward, CO 80481",40.0787237,-105.5026433,3.8,56,'American restaurant',Google,40.0787237-105.5026433
254574,Asian Cuisine # 2,"Asian Cuisine # 2, 302 N Federal Blvd, Riverto...",43.0269754,-108.380989,4.3,38,"'Restaurant', 'Delivery Restaurant'",Google,43.0269754-108.380989
254575,Rosa’s Mexican Grill Chandler,"Rosa’s Mexican Grill Chandler, 3002 N Arizona ...",33.3510567,-111.8429949,4.4,88,'Mexican restaurant',Google,33.3510567-111.8429949
254576,Domino's Pizza,"Domino's Pizza, 804 N Federal Blvd, Riverton, ...",43.0315264,-108.3816407,4.1,178,"'Pizza delivery', 'Delivery Restaurant', 'Take...",Google,43.0315264-108.3816407


## Exportamos el dataset unificado de business


In [22]:
# prompt: exportar a la nube en formato parquet

# Import necessary libraries
from google.cloud import storage

# Define the bucket and file names
bucket_name = "etl_archivos"
file_name = "BusinessUnificados.parquet"

# Create a storage client
client = storage.Client()

# Get the bucket
bucket = client.get_bucket(bucket_name)

# Create a new blob in the bucket
blob = bucket.blob(file_name)

# Convert the DataFrame to parquet format
df_business_unificado.to_parquet("temp.parquet")

# Upload the parquet file to the bucket
blob.upload_from_filename("temp.parquet")

# Delete the temporary file
!rm temp.parquet


## Cargamos el archivo de Reviews de Google

In [ ]:
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'MassachusettsGoogleReviews.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_mgr = pd.read_csv(io.BytesIO(file_content))

df_mgr['user_id'] = df_mgr['user_id'].astype(str)
# mostrar el DataFrame
df_mgr.head()

,user_id,rating,text,gmap_id,platform
0,1.1803396075403138e+20,5,People run it make sure you are comfortable,0x89e4a103dd188585:0xfadc47938337579f,Google
1,1.1016725077517186e+20,5,When you need a helping hand.,0x89e4a103dd188585:0xfadc47938337579f,Google
2,1.1221360671460452e+20,5,Very Helpful and understanding people definite...,0x89e4a103dd188585:0xfadc47938337579f,Google
3,1.0817449247256507e+20,5,Very good place nice things but on Saturday th...,0x89e4a103dd188585:0xfadc47938337579f,Google
4,1.0899013265898385e+20,5,Awesome place,0x89e4a103dd188585:0xfadc47938337579f,Google


## Generamos el business_id a partir de la latitud y la longitud del otro archivo

In [ ]:
df_join = pd.merge(df_mgr, df_md, how='inner', on='gmap_id')
df_join['latitude'] = df_join['latitude'].astype(str)
df_join['longitude'] = df_join['longitude'].astype(str)
df_join['business_id'] = df_join['latitude'] + df_join['longitude']
df_join.drop(columns=['name','address','platform_y','latitude','longitude','category','avg_rating','num_of_reviews','gmap_id'], inplace = True)
df_join.drop_duplicates(inplace=True)
df_join.head()


,user_id,rating,text,platform_x,business_id
0,1.0532458711744035e+20,4,What a great experience. I tried the chicken t...,Google,42.559072-70.881542
1,1.0859745719720491e+20,5,"The owner, T, is a wonderful man. Super friend...",Google,42.559072-70.881542
2,1.1183954618982552e+20,5,"Love this place, always have & always will. ...",Google,42.559072-70.881542
3,1.1498631393680748e+20,5,Awesome pizzas and subs. Their hawaiin pizza i...,Google,42.559072-70.881542
4,1.0126531419117824e+20,1,"This was the worst experience ever, they told...",Google,42.559072-70.881542


## Cargamos el tips de YELP

In [ ]:
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'tipsYelp.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_ty = pd.read_csv(io.BytesIO(file_content))

# mostrar el DataFrame
df_ty.head()



,user_id,business_id,text,platform
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,Yelp
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,Yelp
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,Yelp
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,Yelp
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,Yelp


## Generamos el business_id a partir de la latitud y la longitud del otro archivo

In [ ]:
df_join_yelp = pd.merge(df_ty, df_by, how='inner', on='business_id')
df_join_yelp['rating'] = 'NaN'
df_join_yelp['latitude'] = df_join_yelp['latitude'].astype(str)
df_join_yelp['longitude'] = df_join_yelp['longitude'].astype(str)
df_join_yelp['business_id'] = df_join_yelp['latitude'] + df_join_yelp['longitude']
df_join_yelp.drop(columns=['address','platform_y','latitude','longitude','categories','city','state','stars','review_count'], inplace = True)
df_join_yelp.drop_duplicates(inplace=True)
df_join_yelp.head()



,user_id,business_id,text,platform_x,rating
0,NBN4MgHP9D3cw--SnauTkA,27.9658653-82.443197,They have lots of good deserts and tasty cuban...,Yelp,NaN
1,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,Turkey and roast beef,Yelp,NaN
2,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,Coffee and baby bread,Yelp,NaN
3,boRCdL8ez95KwzWe69VoUg,27.9658653-82.443197,Such a great place to come and get a Cuban san...,Yelp,NaN
4,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,My last time. Continental and a Cuban,Yelp,NaN


#Unimos los dos datasets de reviews

In [ ]:
# Unificar los dataframes uno debajo del otro
df_reviews_unificado = pd.concat([df_join_yelp, df_join], ignore_index=True)

df_reviews_unificado

,user_id,business_id,text,platform_x,rating
0,NBN4MgHP9D3cw--SnauTkA,27.9658653-82.443197,They have lots of good deserts and tasty cuban...,Yelp,NaN
1,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,Turkey and roast beef,Yelp,NaN
2,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,Coffee and baby bread,Yelp,NaN
3,boRCdL8ez95KwzWe69VoUg,27.9658653-82.443197,Such a great place to come and get a Cuban san...,Yelp,NaN
4,DWryovIM4VF2vim4p6WYaw,27.9658653-82.443197,My last time. Continental and a Cuban,Yelp,NaN
...,...,...,...,...,...
866564,1.159423870758638e+20,42.0806616-70.9304844,NaN,Google,5
866565,1.140124865891358e+20,42.0806616-70.9304844,NaN,Google,5
866566,1.019629264472351e+20,42.0806616-70.9304844,NaN,Google,5
866567,1.0392610124987267e+20,42.0806616-70.9304844,NaN,Google,3


## Cargamos el dataset unificado a la nube, en parquet para que pueda ser subido a BigQuery

In [ ]:
# prompt: TypeError: binary file expected, got text file

import io

# Create a StringIO object to hold the DataFrame in memory
sio = io.BytesIO()

# Write the DataFrame to the StringIO object in Parquet format
df_reviews_unificado.to_parquet(sio)

# Create a client object for interacting with Cloud Storage
client = storage.Client()

# Create a bucket object for the desired bucket
bucket = client.get_bucket('etl_archivos')

# Create a blob object for the desired file
blob = bucket.blob('ReviewsUnificados.parquet')

# Upload the DataFrame from the StringIO object to the blob
blob.upload_from_string(sio.getvalue())


## Cargamos el archivo users del dataset de YELP


In [ ]:
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'users.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_uy = pd.read_csv(io.BytesIO(file_content))
df_uy['platform'] = 'Yelp'
# mostrar el DataFrame
df_uy.head()

,user_id,review_count,average_stars,fans
0,qVc8ODYU5SZjKXVBgXdI7w,585,3.91,267
1,j14WgRoU_-2ZE1aw1dXrJg,4333,3.74,3138
2,2WnXYQFK0hXEoTxPtV2zvg,665,3.32,52
3,SZDeASXq7o05mMNLshsdIA,224,4.27,28
4,hA5lMy-EnncsH4JoR-hFGQ,79,3.54,1


In [ ]:
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'Reviews_por_estado.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_ry = pd.read_csv(io.BytesIO(file_content))

# mostrar el DataFrame
df_ry





,state,num_reviews
0,DE,75950
1,MO,339402
2,IL,70445
3,CA,153053
4,AZ,321061
5,LA,323053
6,NJ,276683
7,FL,824870
8,NV,256647
9,ID,150367


## No hay reseñas en Massachusets en este ultimo archivo, por el momento no lo utilizaremos